In [ ]:
%%pyspark
from pyspark.sql.functions import col

iowa2022 = spark.table("iowa.iowa2022").select(col("County"), col("City")).limit(1)
iowa2022.createOrReplaceTempView("iowa2022")

iowa2022.show()

In [ ]:
%%spark

val iowa2021 = spark.read.synapsesql("sqldedicated.dbo.iowa2021").select($"County", $"City").limit(1)
iowa2021.createOrReplaceTempView("iowa2021")

iowa2021.show()

In [ ]:
%%pyspark

iowa2019 = spark.read.load(
    'abfss://[ADLS address].dfs.core.windows.net/IowaLiquorSalesdf2019.parquet', 
    format='parquet'
    ).select(col("County"), col("City")).limit(1)
iowa2019.createOrReplaceTempView("iowa2019")

iowa2019.show()

In [ ]:
import pkg_resources
for pckg in pkg_resources.working_set:
          print(pchg)

In [24]:
import adal

def SqlServerlessAccess():
    tenant_id = "00"
    authority = "https://login.windows.net/" + tenant_id
    resource_app_id_url = "https://database.windows.net/"
    service_principal_id = "8c"
    service_principal_secret = "kV"
    url = "jdbc:sqlserver://{serverless-endpoint-name}-synapse-ondemand.sql.azuresynapse.net"
    database_name = "sqlserverless"
    db_table = "dbo.iowa2020" 
    encrypt = "true"
    host_name_in_certificate = "*.database.windows.net"
    context = adal.AuthenticationContext(authority)
    token = context.acquire_token_with_client_credentials(resource_app_id_url, 
                                                            service_principal_id, 
                                                            service_principal_secret)
    access_token = token["accessToken"]

    return url, db_table, access_token, encrypt, database_name, host_name_in_certificate

url, db_table, access_token, encrypt, database_name, host_name_in_certificate = SqlServerlessAccess()

StatementMeta(efsynapsespark, 27, 25, Finished, Available)

In [ ]:
iowa2020 = spark.read.format("com.microsoft.sqlserver.jdbc.spark") \
              .option("url", url) \
              .option("dbtable", db_table) \
              .option("accessToken", access_token) \
              .option("encrypt", encrypt) \
              .option("databaseName", database_name) \
              .option("hostNameInCertificate", host_name_in_certificate) \
              .load() \
              .select(col("County"), col("City")).limit(1)
iowa2020.createOrReplaceTempView("iowa2020")

iowa2020.show()

In [ ]:
%%sql
SELECT * FROM iowa2022
UNION ALL
SELECT * FROM iowa2021
UNION ALL
SELECT * FROM iowa2020
UNION ALL
SELECT * FROM iowa2019

In [ ]:
%sql
WITH CTE_main
AS(
    SELECT TOP 1 [County], [City]
    FROM [iowa].[dbo].[iowa2022] --SPARK TABLE

    UNION ALL

    SELECT TOP 1 [County] COLLATE Latin1_General_100_BIN2_UTF8, [City] COLLATE Latin1_General_100_BIN2_UTF8
    FROM [sqlserverless].[dbo].[iowa2020] --SERVERLESS DB

    UNION ALL

    SELECT TOP 1 [County], [City]
    FROM OPENROWSET(
        BULK 'abfss://[ADLS].dfs.core.windows.net/IowaLiquorSalesdf2019.parquet', 
        FORMAT = 'parquet') AS ROWS --DATA LAKE PARQUET FILE
)

SELECT * FROM CTE_main